<a href="https://colab.research.google.com/github/Deepika1454/Audio-Classification-with-Pre-trained-Model/blob/main/NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install spacy pymupdf gradio
!python -m spacy download en_core_web_sm
!pip install python-docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 33.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 3.9 MB/s eta 0:00:00


In [6]:
import spacy
import fitz  # PyMuPDF for extracting text from PDFs
import gradio as gr
import pandas as pd
from docx import Document

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

def extract_text(file_path):
    """Extract text from PDF, TXT, DOCX, or CSV files."""
    if file_path.endswith(".pdf"):
        return extract_text_from_pdf(file_path)
    elif file_path.endswith(".txt"):
        return extract_text_from_txt(file_path)
    elif file_path.endswith(".docx"):
        return extract_text_from_docx(file_path)
    elif file_path.endswith(".csv"):
        return extract_text_from_csv(file_path)
    else:
        return "Unsupported file format. Please upload a PDF, TXT, DOCX, or CSV."

def extract_text_from_pdf(pdf_path):
    """Extract text from a PDF file."""
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text("text") + "\n"
    return text.strip()

def extract_text_from_txt(txt_path):
    """Extract text from a TXT file."""
    with open(txt_path, "r", encoding="utf-8") as file:
        return file.read().strip()

def extract_text_from_docx(docx_path):
    """Extract text from a DOCX file."""
    doc = Document(docx_path)
    return "\n".join([para.text for para in doc.paragraphs]).strip()

def extract_text_from_csv(csv_path):
    """Extract text from a CSV file (assuming first column has text)."""
    df = pd.read_csv(csv_path)
    return " ".join(df.iloc[:, 0].astype(str)).strip()  # Convert first column to text

def perform_ner(text):
    """Perform Named Entity Recognition (NER) and return results."""
    doc = nlp(text)
    entities = [(ent.text, ent.label_, spacy.explain(ent.label_)) for ent in doc.ents]

    if not entities:
        return "No named entities found in the document."

    result = "\n🔹 **Named Entities Found:**\n"
    for entity, label, explanation in entities:
        result += f"- {entity} → **{label}** ({explanation})\n"
    return result


In [7]:
def process_pdf(file_path):
    """Extract text from uploaded PDF and perform NER."""
    text = extract_text_from_pdf(file_path)
    return perform_ner(text)

# ✅ Use `type="filepath"` instead of `type="file"`
gr.Interface(
    fn=process_pdf,
    inputs=gr.File(type="filepath"),  # Corrected input type
    outputs="text",
    title="📄 PDF Named Entity Recognition",
    description="Upload a PDF file, and this tool will extract text and perform Named Entity Recognition (NER) using spaCy."
).launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4401f8ba9387699492.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
